# Oasis FM Test Tool v0.6

This notebook allows example insurance structures to be input in OED format and ran against the development version of the Oasis finanical engine. 

## Included

The following terminology is defined in OED. Please visit https://github.com/Simplitium/OED for more information.

* Standard direct insurance modelling 
    * Site (Coverages: Buildings, Other, Contents, BI, PD and All)
        * Ded, Limit
    * Cond (Coverages: All) A single condition per site only.
        * MinDed, Limit
    * Pol (Coverages: All)
        * Ded, MinDed, MaxDed, Limit
    * Layers - Multiple per account
        * Attachment, Limit, Participation
    * Deductible Types 0 and 2 (% TIV) only
    * Limit Types 0 and 2 (% TIV) only
    

## Not included
* Peril codes
* Type 1 terms (% loss)
* Code field


## Test cases

A selection of test cases can be found in the examples folder.


## Input files 

Files must be named the following:

* account.csv
* location.csv


## Validation Rules

* TBC

In [46]:
# 1. Notebook and Python setup
%config IPCompleter.greedy=True

# Standard Python libraries
import io
import json
import os
import shutil

# 3rd party Python libraries
import jupyter_helper
import pandas as pd
import six
    
# Oasis imports
import oasislmf._data as mdk_static_data 
from oasislmf.model_preparation import oed
from oasislmf.utils.deterministic_loss import (
    generate_oasis_files,
    generate_losses,
)

In [ ]:
jupyter_helper.file_uploader('examples/uploaded')

In [47]:
# 2. Load the OED for a worked example, in this case FM Test Case 3.

# Note that only the currently used fields are shown unless show_all is set to True. 
oed_dir = os.path.abspath('ftest/fm3')

# Account file
oed_account_file = os.path.join(oed_dir, "account.csv")
if not os.path.exists(oed_account_file):
    print("Path does not exist: {}".format(oed_account_file))
    exit(1)
account_df = pd.read_csv(oed_account_file)

# Location file
oed_location_file = os.path.join(oed_dir, "location.csv")
if not os.path.exists(oed_location_file):
    print("Path does not exist: {}".format(oed_location_file))
    exit(1)
location_df = pd.read_csv(oed_location_file)

In [48]:
# 3. View/edit the account data. 
account_grid = jupyter_helper.show_df(account_df)
account_grid

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

In [49]:
# 4. View/edit the location data.
location_grid = jupyter_helper.show_df(location_df)
location_grid

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

In [50]:
# 5. Create the direct Oasis files directory `run_reinsurance`, and set up
# the MDK provided static source -> canonical CSV transformation files
# for the source exposure and account files.
direct_oasis_files_dir = os.path.abspath('run_reinsurance')
if os.path.exists(direct_oasis_files_dir):
    shutil.rmtree(direct_oasis_files_dir)

os.mkdir(direct_oasis_files_dir)

mdk_static_data_fp = mdk_static_data.__path__._path[0]
srcacctocan_trans_fp = os.path.join(mdk_static_data_fp, 'MappingMapToOED_CanAccA.xslt')
srcexptocan_trans_fp = os.path.join(mdk_static_data_fp, 'MappingMapToOED_CanLocA.xslt')

In [51]:
# 6. Pick up any edits in the grid before running the analysis
# and copy the updated files into `run_reinsurance`
account_df = account_grid.get_changed_df()
location_df = location_grid.get_changed_df()

account_df.to_csv(path_or_buf=os.path.join(direct_oasis_files_dir, "account.csv"), encoding='utf-8', index=False)
location_df.to_csv(path_or_buf=os.path.join(direct_oasis_files_dir, "location.csv"), encoding='utf-8', index=False)

In [52]:
# 7. Generate the direct Oasis files in `run_reinsurance`
output_dir = os.path.join(direct_oasis_files_dir, "output")
(ri_layers, xref_descriptions) = generate_oasis_files(
    direct_oasis_files_dir, output_dir, 
    srcexptocan_trans_fp, srcacctocan_trans_fp)


In [53]:
# 8. Generate the direct losses
loss_factor = 0.5
net_losses = generate_losses(
    output_dir,
    xref_descriptions,
    loss_percentage_of_tiv=loss_factor,
    ri_layers=ri_layers
)

In [54]:
# 9. View the direct losses.
direct_losses = net_losses.get('direct') or net_losses.get('Direct')
direct_losses

,portfolio_number,policy_number,account_number,location_number,location_group,cedant_name,producer_name,lob,country_code,reins_tag,coverage_type_id,peril_id,tiv,loss_gul,loss_il
0,1,1,1,1,None,None,None,None,US,None,1,1,1000000,1000000.0,900000.0
1,1,1,1,1,None,None,None,None,US,None,2,1,100000,100000.0,90000.0
2,1,1,1,1,None,None,None,None,US,None,3,1,50000,50000.0,45000.0
3,1,1,1,1,None,None,None,None,US,None,4,1,20000,20000.0,18000.0


## Optional Steps


In [55]:
# 10. Write the direct losses to file (in `run_reinsurance`)
direct_losses.to_csv(path_or_buf=os.path.join(direct_oasis_files_dir, 'output', 'output_direct.csv'), encoding='utf-8', index=False)